## Macromolecule density

This notebook assists to generate 3D density mpas for macromolecules. First part allows to download atomic models from PDB, this step is **optional**, PDB models can also be processed from disk. Second, the 3D density map is generated from the PDB file and stored as MRC file. 

In [1]:
from core.vtk_utilities import *
from core.utilities import *
from core.widgets_utilities import *
from core.tk_utilities import *
import io

## Download PDB

PDB atomic models can be downloaded following next steps. You can **skip** this step if you already have the PDB model:

1. In the text box enter the name of the protein you want to download.
2. Click the **"Select"** button to select the path where to save the files or leave it empty and they will be saved in a project directory.
3. After filling it click on the **"Save"** button to download the proteins from the protein data bank and a vtk window will appear to display the downloaded protein.

**Note** that you can repeat this process as many times as you want.

In [3]:
DEFAULT_DESTINATION_PATH = "../data/templates/pdbs/"

def save(btn):
    with output_widget:
        path = check_dir(file_destination_widget.value, DEFAULT_DESTINATION_PATH)
        path = download_pdb(text_protein_widget.value, path)
        if text_protein_widget.value:
            visualize_pdb(path, 800, 600)
            text_protein_widget.value = ""
        window_download_p(path)

text_protein_widget, save_button, file_destination_widget = widgets_visualize_pdb()
output_widget = widgets.Output()
save_button.on_click(save)
display(output_widget)

FileChooser(path='/home/martinez/workspace/pycharm_proj/polnet/gui', filename='', title='Select the path and t…

Output()

## Transform to density maps

Once the PDBs are downloaded you can transform them into density maps. The files you want to transform can be one or several and are saved with the same name and .mrc extension in the indicated directory or, if none is indicated, in the default directory.

- Intput parameters:

    - Voxel size: scale (Angstrom/voxel) of the tomogram resolution. A higher value it means more fine details will be rendered, and a lower resolution means coarser rendering.
    - Resolution: equivalent to Gaussian lowpass with 1/e width at 1/resolution. This value must be at least 2x higher than voxel size.
    - Offset: additional offset in voxels for the tomogram size. Add space to the tomogram box.

- Advanced parameters, if you not sure just keep the default values:

    - Het: flag indicating whether HET atoms should be included in the map. HETATMs represent heteroatoms, which are atoms that are not part of the main chain.
    - Selected chains: this is a list of protein chains. A protein chain is a continuous sequence of amino acids that forms part of the structure of a protein. Strings are usually labeled with letters, such as A, B, C, etc. For example an entry in the pdb file consists of:
        - *ATOM      1  N   ASP A   1      13.924  41.074  28.675  1.00 15.00*, where N and A repesent the chain identifier
    - Model: process only the model corresponding to that number instead of all. None value for all.

If you want to save some in one directory and others in another, just run with the files you want for your path and run again with the rest for the other path

**Note** that this process may take several minutes, and it is specially costly for large macromolecules such as ribosomes.


In [5]:
DEFAULT_DESTINATION_PATH2 = "../data/templates/mrcs_10A/"
files_path = []

def select(btn):
    global files_path
    select_files(files_path, ".pdb")
    replace_options(files_path, dropdown)

def convert(btn):
    global files_path
    with output_widget2:
        if chains_widget.value:
            chains = chains_widget.value.split(',')
        else:
            chains = None
        if use_model_checkbox.value == False:
            m = None
        else:
            m = model_widget.value
        path = check_dir(file_destination_widget.value, DEFAULT_DESTINATION_PATH2)
        convert_to_mrc(files_path, path, apix_widget.value, res_widget.value, offset_widget.value*apix_widget.value, het_widget.value, chains, m)
        files_path = []

apix_widget, res_widget, offset_widget, het_widget, chains_widget, use_model_checkbox, model_widget, files_button, dropdown, file_destination_widget, exec_button = widgets_convert_to_mrc()
exec_button.on_click(convert)
files_button.on_click(select)
output_widget2 = widgets.Output()
display(output_widget2)

Output()